# Epicurious

www.epicurious.com - a food recipe website with over 330,000+ recipes

This notebook performs webscraping on epicurios website to collect - Most Reviewed Food recipes for over 900 webpages of top rated recipe lists

- Recipe Title
- Recipe Link
- Average Rating of the Recipe
- Total Number of Reviews for the Recipe

Information regarding more recipes can be collected by changing the *numberOfPagesToCollect*

In [ ]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [ ]:
baseURL = 'https://www.epicurious.com/search/?sort=mostReviewed&page='
epicuriousURL = 'https://epicurious.com'

In [ ]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
headers = {"User-Agent": user_agent}

In [ ]:
numberOfPagesToCollect = input('Enter the count of the number of pages to collect the recipe lists on Epicurious: ')

In [ ]:
start_time = time.time()
epicurious_recipe_title_link = []
columns = ['Recipe Title', 'Recipe Link', 'Recipe Average Rating', 'Recipe Rating Count']        

count = 1

for page in range(1, numberOfPagesToCollect):
    
    response = requests.get(baseURL + f'{page}', headers = headers)
    html_content = response.content
    doc = BeautifulSoup(html_content, 'html.parser')
    print(count)
    
    count += 1
    
    recipe_table_tags = doc.find_all("div", class_="results-group")[0]
    page_one_recipes = recipe_table_tags.find_all("article", class_="recipe-content-card")
    
    
    for recipe in page_one_recipes:
        recipe_details = recipe.find("a", class_="photo-link")
        recipeTitle = recipe_details.get("aria-label")
        recipeLink = epicuriousURL + recipe_details.get("href")
        recipeSummary_tag = recipe.find("header", class_="summary")
#         authorSource_tag = recipeSummary_tag.find("strong", class_="tag")
#         authorName = authorSource_tag.find("span", class_="recipe-source").text.replace('from ', '')
        recipeRatingsSummary_tag = recipeSummary_tag.find("dl", class_="recipes-ratings-summary")
        recipeRatingsCount = recipeRatingsSummary_tag.get("data-reviews-count")
        recipeAverageRating = recipeRatingsSummary_tag.get("data-reviews-rating")
        
        epicurious_recipe_title_link.append((recipeTitle, recipeLink, recipeAverageRating, recipeRatingsCount))

    if ((count%50) == 0):
        epicurious_recipeLinks_df = pd.DataFrame(epicurious_recipe_title_link, columns = columns)

epicurious_recipeLinks_df = pd.DataFrame(epicurious_recipe_title_link, columns = columns)

end_time = time.time()

total_time = end_time - start_time
print(f"Total Time take = {total_time} seconds")
# epicurious_recipeLinks_df.head()

In [ ]:
epicurious_recipeLinks_df.to_csv('./epicuriousRecipeLinks.csv', index = None)